In [136]:
#from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
#nltk.data.path.append('/home/dada/jupyter/bobo/all/')
#nltk.download('punkt')
import xml.etree.ElementTree as ET
from lxml import etree
parser = etree.XMLParser(recover=True)
import os
data_path = 't2-doc/'

In [121]:
data = {}
for xml in os.listdir(data_path):
    with open(data_path + xml) as f:
        index = int(xml.split('.xml')[0])
        data[index] = {}
        #print(index)
        tmp = f.read().split('\n\n')
        #date = etree.fromstringlist(['<date>', tmp[0], '</date>'], parser=parser)[0].text
        title = etree.fromstringlist(['<title>', tmp[0], '</title>'], parser=parser)[0].text
        try:
            abstract = etree.fromstringlist(['<abstract>', tmp[1], '</abstract>'], parser=parser)[0].text
        except:
            abstract = etree.fromstringlist(['<abstract>', tmp[2], '</abstract>'], parser=parser)[0].text
        #print(title)
        #print(abstract)
        #data[index]['date'] = date
        data[index]['title'] = title
        data[index]['abstract'] = abstract
        

In [122]:
alltitle = [data[i]['title'].strip()+'.' for i in sorted(data)]
allabs = [data[i]['abstract'].strip() for i in sorted(data)]
#alldata = [a+b for a,b in zip(alltitle, allabs)]

In [123]:
#D2V
#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(alldata)]
tagged_title = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(alltitle)]
tagged_abs = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(allabs)]
max_epochs = 100
vec_size = 64
alpha = 0.025

model_title = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=5,
                workers=10,
                dm =1)
  
model_title.build_vocab(tagged_title)

model_abs = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=5,
                workers=10,
                dm =1)
  
model_abs.build_vocab(tagged_abs)



for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model_title.train(tagged_title,
                total_examples=model_title.corpus_count,
                epochs=model_title.iter)
    model_abs.train(tagged_abs,
                total_examples=model_abs.corpus_count,
                epochs=model_abs.iter)
    # decrease the learning rate
    model_title.alpha -= 0.0002
    model_abs.alpha -= 0.0002
    # fix the learning rate, no decay
    model_title.min_alpha = model_title.alpha
    model_abs.min_alpha = model_abs.alpha

#model.save("d2v.model")
#print("Model Saved")

/home/dada/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/home/dada/.local/lib/python3.5/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
/home/dada/.local/lib/python3.5/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [124]:
import networkx as nx
G = nx.read_edgelist('t2-train.txt', nodetype=int, create_using=nx.DiGraph())

#negative sampling
neg = []
for node in G:
    for nbr, datadict in G.adj[node].items():
        for nnbr, datadict in G.adj[nbr].items():
            if nnbr not in G.adj[node]:
                neg.append(np.array([node, nnbr]))
neg = np.array(neg)
print(neg.shape)

In [126]:
np.random.shuffle(neg)
pos_num = sum([1 for line in open('t2-train.txt')])
neg_sample = neg[:pos_num]
#build embeddings dict
embeddings = []
for i in range(len(tagged_data)):
    title_emb = model_title.infer_vector(alltitle[i])
    abs_emb = model_abs.infer_vector(allabs[i])
    embeddings.append(np.hstack((title_emb, abs_emb)))
    #embeddings.append(model.infer_vector(alldata[i]))
embeddings = np.array(embeddings)
print(embeddings.shape)

(17500, 128)


In [127]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier

In [128]:
X = []
y = []
with open('t2-train.txt') as f:
    for line in f:
        id1, id2 = map(int, line.split())
        X.append(np.hstack((embeddings[id1-1], embeddings[id2-1])))
        y.append(1)
    for id1, id2 in neg_sample:
        X.append(np.hstack((embeddings[id1-1], embeddings[id2-1])))
        y.append(0)
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=33)

In [130]:
params = {
    'eta': 0.4,
    'max_depth': 4,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'seed': i,
    'silent': False,
    'nthreads':8
}
dtrain = xgb.DMatrix(X_train, label=y_train)
watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'val')]
model = xgb.train(params, dtrain, 1000, watchlist, verbose_eval=True, early_stopping_rounds=10)

[0]	train-logloss:0.684898	val-logloss:0.686257
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 10 rounds.
[1]	train-logloss:0.678662	val-logloss:0.681203
[2]	train-logloss:0.675092	val-logloss:0.677735
[3]	train-logloss:0.671338	val-logloss:0.674592
[4]	train-logloss:0.668988	val-logloss:0.672871
[5]	train-logloss:0.665417	val-logloss:0.66974
[6]	train-logloss:0.663728	val-logloss:0.668619
[7]	train-logloss:0.662071	val-logloss:0.66741
[8]	train-logloss:0.659928	val-logloss:0.665812
[9]	train-logloss:0.658473	val-logloss:0.664365
[10]	train-logloss:0.657137	val-logloss:0.663836
[11]	train-logloss:0.655422	val-logloss:0.662183
[12]	train-logloss:0.654242	val-logloss:0.661326
[13]	train-logloss:0.652435	val-logloss:0.660105
[14]	train-logloss:0.651369	val-logloss:0.659105
[15]	train-logloss:0.649234	val-logloss:0.657405
[16]	train-logloss:0.647911	val-logloss:0.656197
[17]	train-logloss:0.646915	val-

In [131]:
#model.best_ntree_limit

In [132]:
#xgbc = XGBClassifier()
#xgbc.fit(X_train, y_train)
#print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))


In [133]:
testing_data = []
with open('t2-test.txt') as f:
    for line in f:
        id1, id2 = map(int, line.split())
        testing_data.append(np.hstack((embeddings[id1-1], embeddings[id2-1])))
testing_data = np.array(testing_data)
pred = model.predict(xgb.DMatrix(testing_data))
mean = np.median(pred)
print(pred)
pred[pred>=mean] = 1
pred[pred<mean] = 0
np.savetxt('pred.txt', pred.reshape(-1, 1), fmt='%d')

[0.8520391  0.59940124 0.5410446  ... 0.51097    0.7069582  0.61034423]


In [134]:
print(np.sum(pred))
print(sum([1 for a in pred]))


38474.0
76947


In [135]:
!python3 pred-txt-to-csv.py pred.txt